# Wayve Technical Test


1. Suppose you have an arbitrary global variable which is a binary 2-dimensional array called M. A “path” to (i, j) is defined as a sequence of right and down steps starting from position (0, 0) to (i, j) going only through positions which contain a 1. Write a function which returns the number of unique paths to an arbitrary position (i, j). By convention, the number of paths to (0, 0) is 1. What is the computational complexity of your algorithm? What is the memory complexity?


### Approach #1
We can think of M as a directed graph, with edges $(i,j) \to (i+1,j)$ (down) and $(i,j) \to (i,j+1)$ (right), if each of these positions contain a 1. The number of ways to each vertex would be the sum of its UP and LEFT neighbor.

In [19]:
def valid(M, i, j):
    """ Returns the value of M (whether there is a 1 at i,j)"""
    height, width = M.shape
    if i < height and j < width and i >= 0 and j >= 0:
        return M[i,j]
    return 0
    
def count_unique_paths(M, destination=(0,0)):
    K = np.ones_like(M)
    for i in range(destination[0]+1):
        for j in range(destination[1]+1):
            if i == 0 and j == 0:
                continue
            K[i,j] = valid(M, i-1, j)*K[i-1,j] + valid(M, i, j-1)*K[i,j-1]
            
            # now, this part is unclear
            # Do we want to say that there are 0 paths if (i,j) contains a 0 in M?
            # if so, comment out this line
            K[i,j] *= valid(M, i, j)
    
    return K[destination[0],destination[1]]


import numpy

M = np.ones((3,3), dtype=np.uint8)
print(count_unique_paths(M, (0,0)), 1)
print(count_unique_paths(M, (2,2)), 6)


M[1,1] = 0 # now, it's a donut shape
print(count_unique_paths(M, (1,1)), 0)
print(count_unique_paths(M, (2,2)), 2)

1 1
6 6
0 0
2 2


**Computational Complexity**: It iterates the entire array, up to $(i,j)$, so the computational complexity scales like $O(i*j)$. This could be linear or quadratic, depending on what you take $n$ to be :)

**Memory Complexity:** We store the result array in-memory, so this also takes $O(i*j)$ memory.

### Sketch of Approach 2
Why might this not be good? If $i$ and $j$ are large, and $M$ is very sparse, we might want our algorithm to 1) run faster, and 2) not allocate all the memory

This requires exploring the nodes in M with a search algorithm like BFS, instead of the dynamic programming type solution we had in Approach 1.

**Computational Complexity**: Could potentially go down to $O(m)$, where $m$ is the number of nonzero entries in $M$

**Memory Complexity**: Could also go down to $O(m)$, where $m$ is the number of nonzero entries in $M$

2. We wish to train a machine learning algorithm on an array, x, containing floating point numbers in the interval [0.0, 1.0). The data is horribly unbalanced, and we wish to filter the dataset so that it contains an equal number of examples from each interval [0 0.1), [0.1, 0.2), ... [0.9, 1.0). Write a function which returns an array containing this filtered set examples from the array x.

In [25]:
# Raw approach: We can go through the array, and count how many are in each interval!
# Then, we resample

# Say our probabilities were [.9, .05, .06...]
# We want to boost it so that the low probabilities get a HIGH chance
# of being selected (like affirmative action)
# (ie, IMPORTANCE SAMPLING)
def balance_with_filter(x):
    # make our table
    counts = np.zeros(10)
    for num in x:
        bucket = math.floor(num*10)
        counts[bucket] += 1
    
    probabilities = counts / sum(counts)
    resampled_probabilities = .1 / probabilities
    normalized_resamples = resampled_probabilities / sum(resampled)
    
    # now we sample?
    dataset = []
    for num in x:
        bucket = math.floor(num*10)
        if random.random() < normalized_resamples[bucket]:
            dataset.append(num)
    return dataset

Thngs to do: check if any of the probabilies are 0 before dividing, etc

Computationally, the it sweeps through the array once, so $O(n)$. I think the bulk of the computation will be in sampling, so if I can think of another approach with *doesn't* involve sampling, I'll let you know!

Memory, here's the good part: it doesn't use any (well, only a tiny bit) memory! $O(1)$ Could potentially use an algorithm like this for on-line streaming

3. Spaceship and target problem

You are given a spaceship and a target, each with position and orientation as 2D vectors.
The spaceship needs to shoot the target, but can only fire within a certain distance $d$ and
angle $\theta$ from its position. Write a function to determine if it can hit its target.

In [33]:
def can_hit(spaceship, target, d, theta):
    s_pos, s_or = spaceship
    t_pos, t_or = target
    
    
    distance = np.sqrt(np.sum((s_pos - t_pos)**2))
    
    # it'll be easier if we convert to complex coordinates
    s_pos = complex(s_pos)
    t_pos = complex(t_pos)
    target_angle = np.angle(t_pos - s_pos)
    orientation_angle = np.angle(s_or)
    
    return distance < d and np.abs(orientation_angle - target_angle) < theta
    

We can also do it without the complex notation :) (by unwrapping to sines and cosines).

# Additional Problem solving

**1. How much data would a fleet of ten autonomous vehicles collect over a week?**

Whoa!!! Say, we have 10 cameras per car. A video file is probably like 10M per minute (with those really cool video codecs), so if we were to stream this for 10 hours a day, it'd be 10M/minute/camera * 60 minutes/hour * 10 hours * 10cameras, or 60GB per car per day. Or, 4200GB for the entire fleet in a week. Wow, that is so much. I bet I"m off though.

**2. Given a cube with edge length x, what is the shortest distance from one vertex to the opposite vertex, travelling only on the cube’s surface?**

$x\sqrt{5}$? I'm imagining unwrapping the cube and using PYthagoras's fancy theorem

**3. Our autonomous vehicle is traveling at a constant speed of 10m/s, 10m directly behind a vehicle which is travelling in the same direction. The vehicle in front applies a constant braking force and comes to a stop after travelling a further 20m. Assuming our AI system has a latency from sensing to action of 0.5 seconds, what is the minimum constant deceleration our AI system must apply to prevent a collision?**

Vehicle in front decelerates from 10 to 0, in t time. It also goes 20m. so $20=10 t + 1/2 a t^2$ and $-10 = at$, so $t=4$

Our vehicle starts to slow 0.5 seconds later, after which it is now 5m + 20m out from the finish line. Let's say to maintain a safe distance we want to be 1m behind the car, which is also 4m long, so 20m. We get 3.5 seconds to decelerate. 

$20 = 10(3.5) + 1/2 a (3.5)^2$

In [10]:
(20 - 35)/(0.5 * 3.5**2)

-2.4489795918367347

In [9]:
-10/4

-2.5

Surprising - we have to slow down less than the car in front of us... perhaps this is because we start 10m behind?

**4. You have 3000 barrels of fuel and a truck which can carry 1000 barrels of fuel at any one time. You want to deliver fuel to a town 1000km away. However, in order to drive 1km, your truck requires 1 barrel of fuel. At every 1km from your position to the town there is a stopping point where you can safely and securely leave as many barrels as you like. What is the maximum number of barrels of fuel you can deliver to the town?**

Let's start with this scenario: Truck drives 100km, deposits 800 barrels, drives back, and loads up, and does this until the entire depot is moved 100km closer to the goal. We're not 900km away, and 600 barrels of fuel short :( This can't work too tell

What if we drive closer? 1km away. We can carry 998 barrels per trip, but still costs us 6 barrels. So moving the entire depot $d$ km has cost us $6d$ barrels.

At some point it's going to take us less than three trips to move the depot, so the cost would decrease. And then my guess would be to wait until you get down to 1000, at which point, you go all the way to the final destination.

**5. You have a fleet of 25 autonomous cars. You can race up to 5 at a time but don’t have a timing device. You simply can observe their relative ordering. What is the minimum number of races required to find the 3 fastest cars without using a stopwatch?**

Hm... We probably have to collect them at some point. More thinking required!

**6. A rabbit is climbing a ladder with 10 rungs. It can jump 1 or 2 rungs at a time. How many different ways can it get to the top?**

$n(10) = n(9) + n(8)$, and so forth. So:

* n(0) = 1
* n(1) = 1
* n(2) = 1 + 1 = 2
* n(3) = 1 + 2 = 3
* n(4,5,6,7,8,9,10) =
* 5, 8, 13, 21, 34, 55, 89 ways (Fibonacci)


**7. What is the expected value of the maximum of two throws of a fair die?**
$$ (1* 1 + 2 * 3 + 3*5 + ... + 6*11)/36$$

In [11]:
sum([i * (2*i-1) for i in range(1,7)])/36

4.472222222222222


**8. You have 100 sheets of paper. The first says, “at most 0 of these statements are true”, the second says “at most 1 of these statements are true”, ... , the last says “at most 99 of these statements are true”. How many of the statements are true?**

If none of the statements are true, then all of them are true. If 10 of them are true, then 90 of them are true... This means that 50 of them have to be true (and the papers saying "at most 50" all the way to "at most 99" are true)

**9. There is an 8x8 chess board in which two diagonally opposite corners have been cut off You are given 31 dominos, and a single domino can cover exactly two squares. Can you use the 31 dominos to cover the entire board? Prove your answer (by providing an example, or showing why it’s impossible).**

Hmm, I've seen this one before. No, because each domino covers a black and a white square!

**10. What is the maximum number of squares can you make on a grid of 5x5 squares?**

We can choose edges to make squares.. 6 choose 2 = 15 combinations on each side, yielding 225 squares. 